In [ ]:
with data as (select * from `bigquery-export-352717.analytics_307522449.events_*`),
unnested as (
  SELECT 
  event_date,
  event_timestamp,
  event_name,
  user_pseudo_id,
  device.category,
  traffic_source.medium,
  traffic_source.source,
  key, 
  value.string_value, 
  value.int_value
  FROM data, UNNEST(event_params) as params),
Str as (
  Select 
    event_date,
    event_timestamp,
    event_name,
    user_pseudo_id,
    category,
    medium,
    source,
    page_location,
    campaign
  from unnested
  pivot (any_value (string_value) for key in ('page_referrer', 'page_location', 'term', 'campaign'))
  ),
inte as (
  select 
  event_timestamp,
  event_name,
  user_pseudo_id,
  firebase_conversion,
  ga_session_id
  from unnested
  pivot (any_value (int_value) for key in ('firebase_conversion', 'ga_session_id'))
  ),
allrows as( select 
    str.event_date,
    str.event_timestamp,
    str.event_name,
    str.user_pseudo_id,
    category,
    medium,
    source,
    page_location,
    campaign,
    inte.firebase_conversion,
    inte.ga_session_id
  from str
  inner join inte on str.event_name=inte.event_name and str.user_pseudo_id=inte.user_pseudo_id and str.event_timestamp=inte.event_timestamp
  where page_location is not null and firebase_conversion is not null
  order by event_timestamp, firebase_conversion desc)